In [1]:
import pandas as pd
import numpy as np
import sys 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix 

Using TensorFlow backend.


In [52]:

    
def get_marital_status(row):
    if row.Sex == 'female':
        return ('Mrs.' in row.Name) | ('Mme.' in row.Name)
    
def get_deck_from_cabin(cabin):
    if len(cabin) > 0:
        deck = cabin[:1]
        decks = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "O": 8, "T": 9, "U": 10}
        return decks[deck]

def get_dataframe(file_name):
    print('Filename: ', file_name)
    df = pd.read_csv(file_name)
    avg_age = df.Age.mean()
    df.Age = df.Age.fillna(avg_age).astype(int)
    df.Name.apply(str)
    df['Married'] = df.apply(get_marital_status, axis=1)
    df.Married = df.Married.astype('bool')
    df['LastName'] = df.Name.apply(lambda name: name.split(', ')[0])
    df.Cabin = df.Cabin.fillna('U0')
#     deck prob has too many unknown values to be useful
    df['Deck'] = df.Cabin.apply(get_deck_from_cabin)
    genders = { 'male': 0, 'female': 1}
    df.Sex.replace(genders, inplace=True) 
    encoder = LabelEncoder()
    df['LastName'] = encoder.fit_transform(df['LastName']).astype(int)
    df['Relatives'] = df.Parch + df.SibSp
    return df
    
def get_predictions(df_train, df_test):
#     Married, Cabin, Relatives, Deck not improve accuracy
    columns = ['Pclass', 'Sex',  'LastName', 'Age']
    X_train = df_train[columns]
    df_train.Survived = df_train.Survived.astype('bool')
    y_train = df_train['Survived']
    X_test = df_test[columns]
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    output=pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Survived': y_pred})
    output.Survived = output.Survived.astype(int)
    return output

In [53]:
df_train = get_dataframe('train.csv')
df_test = get_dataframe('test.csv')
predictions = get_predictions(df_train, df_test).to_csv('submission.csv', index=False)
print('Done')

Filename:  train.csv
Filename:  test.csv
Done
